## 1. Update all Data

In [9]:
# %load ../src/data/get_data.py
import subprocess
import os
import sys
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime



def get_johns_hopkins():

    git_pull = subprocess.Popen( "/usr/bin/git pull" ,
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ),
                     shell = True,
                     stdout = subprocess.PIPE,
                     stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():

    data = requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object = json.loads(data.content)

    full_list = []

    for pos, each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])


    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()


Error : b'From https://github.com/CSSEGISandData/COVID-19\n   c955694e..0314c8be  web-data   -> origin/web-data\n'
out : b'Already up to date.\n'
 Number of regions rows: 16


## 2. Process Pipleline

In [10]:
# %load ../src/data/process_JH_data.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plot
from datetime import datetime

pd.set_option('display.max_rows',500)

def store_relational_JH_data():

    data_path = '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw = pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country','Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))


if __name__ == '__main__':

    store_relational_JH_data()


 Number of rows stored: 63042


## Slope Calculation